In [60]:
import pandas as pd
import numpy as np

In [ ]:
data_dict = {}

names = ["Brazil", "Colombia", "Indonesia", "Vietnam", "Honduras"]
for name in names:
    file_path = f"data/weather_data_{name}.csv"
    data_dict[f"{name}_data"] = pd.read_csv(file_path)


In [129]:
for name in names:
    df = data_dict[f"{name}_data"]
    
    # Convert to datetime and extract date parts
    df["date"] = pd.to_datetime(df["date"]).dt.date
    df["month"] = pd.to_datetime(df["date"]).dt.month
    df["year"] = pd.to_datetime(df["date"]).dt.year
    
    # Create formatted "year_month" column
    df["year_month"] = df.apply(lambda row: f"{row['month']:02d}/{row['year']:02d}", axis=1)

    numeric_cols = df.select_dtypes(include=np.number).columns
    numeric_cols = [col for col in numeric_cols if col not in ["month", "year"]]
    
    grouped_mean = df.groupby("year_month")[numeric_cols].mean().reset_index()
    
    grouped_mean["year_month"] = pd.to_datetime(grouped_mean["year_month"], format="%m/%Y")
    
    # Store result
    data_dict[f"{name}_weather_grouped"] = grouped_mean
    cols = ["daylight_duration", "sunshine_duration"]
    data_dict[f"{name}_weather_grouped"][cols] = data_dict[f"{name}_weather_grouped"][cols].apply(lambda row: row / 3600 if row.max() > 24 else row, axis=1)



In [128]:
names = ["Brazil", "Colombia", "Indonesia", "Vietnam", "Honduras"]
for name in names:
    df = data_dict[f"{name}_weather_grouped"]
    output_file = f"data/{name}_weather_cleaned.csv"
    df.to_csv(output_file, index=False)
